In [9]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import glob

In [11]:
import numpy as np

# Create an array with float numbers and NaNs
arr = np.array([np.nan, np.nan,5.0])

print(array_with_nans)


[nan nan  5.]


In [12]:
np.sum(arr)

nan

In [6]:
def generate_the_99_quantile(prec_filtered,latt,lonn):

    arr_the = np.nanquantile(prec_filtered, 0.99, axis=0)
    ds_the = xr.Dataset({'p': ([ 'lat', 'lon'], arr_the)},
                    coords={'lon': (['lon'], lonn),
                            'lat': (['lat'], latt),})
    return ds_the


In [7]:
def create_max_datasets(arr_ep, latt, lonn):

    arr_freq = arr_ep.reshape(43, 92, 24, arr_ep.shape[1], arr_ep.shape[2])
    arr_maxp = np.nanmax(arr_freq, axis=(1, 2))
    arr_maxh = np.nanmax(arr_freq, axis=1)
    arr_maxh = np.nan_to_num(arr_maxh, nan=-1)
    arr_maxh_indices = np.nanargmax(arr_maxh, axis=1)
    arr_maxh_indices = arr_maxh_indices.astype(float)
    nan_mask = np.isnan(arr_maxp)
    arr_maxh_indices[nan_mask] = np.nan
    
    ds_maxp = xr.Dataset(
        {'p': (['time', 'lat', 'lon'], arr_maxp)},
        coords={
            'lon': (['lon'], lonn),
            'lat': (['lat'], latt),
            'time': ('time', np.arange(1980, 2023))
        }
    )

    ds_maxh = xr.Dataset(
        {'h': (['time', 'lat', 'lon'], arr_maxh_indices)},
        coords={
            'time': (['time'], np.arange(1980, 2023)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )
    
    
    arr_h = np.nanmean(~np.isnan(arr_freq), axis=1)*92

    ds_hour_count = xr.Dataset(
        {'c': (['year', 'hour', 'lat', 'lon'], arr_h)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'hour': (['hour'], np.arange(0, 24)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_maxp, ds_maxh, ds_hour_count

In [8]:
def create_season_mean_dataset(arr_ep, latt, lonn):

    arr_freq = arr_ep.reshape(43, 92, 24, arr_ep.shape[1], arr_ep.shape[2])
    arr_season = np.nanmean(arr_freq, axis=1)
    
    ds_season_mean = xr.Dataset(
        {'p': (['year', 'hour', 'lat', 'lon'], arr_season)},
        coords={
            'year': (['year'], np.arange(1980, 2023)),
            'hour': (['hour'], np.arange(0, 24)),
            'lat': (['lat'], latt),
            'lon': (['lon'], lonn)
        }
    )

    return ds_season_mean

In [79]:
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'

folder_names = [
  
    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 
    'U-87', 'U-88'
]


for folder in folder_names:
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    full_path = os.path.join(base_path, folder, file_pattern)
    all_files = glob.glob(full_path)
    #####
    summer_files = [f for f in all_files if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_summer = xr.open_mfdataset(summer_files)
    ds_jja = ds_summer.sel(time=ds_summer['time'].dt.month.isin([6, 7, 8]))
    lonn = ds_jja.lon.values
    latt = ds_jja.lat.values
    prec = ds_jja.p.values
    prec_filtered = np.where(prec >= 0.1, prec, np.nan)
    arr_the = np.nanpercentile(prec_filtered, 99, axis=0)
    arr_ep = np.copy(prec_filtered)
    arr_ep = np.where(prec_filtered >= arr_the, prec_filtered, np.nan)
    ds_the = generate_the_99_quantile(prec_filtered, latt, lonn)
    ds_maxp, ds_maxh, ds_hour_count = create_max_datasets(arr_ep, latt, lonn)
    ds_season_mean = create_season_mean_dataset(arr_ep, latt, lonn)
    output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
    ds_the.to_netcdf(output_folder+'prec_the_'+folder+'.nc')
    ds_maxp.to_netcdf(output_folder+'prec_maxp_'+folder+'.nc')
    ds_maxh.to_netcdf(output_folder+'prec_maxh_'+folder+'.nc')
    ds_hour_count.to_netcdf(output_folder+'hour_count_'+folder+'.nc')
    ds_season_mean.to_netcdf(output_folder+'season_mean_'+folder+'.nc')

2024-10-07 10:11:52
2024-10-07 10:19:08
2024-10-07 10:25:55
2024-10-07 10:32:10
2024-10-07 10:38:26
2024-10-07 10:44:29
2024-10-07 10:50:45
2024-10-07 10:56:36
2024-10-07 11:03:07


In [77]:
summer_files

[]

In [78]:
full_path

'/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/U-79/PREC_ACC_NC.wrf2d_d01_????-??-??.nc'